### 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [91]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd
from transliterate import translit
from pymongo import MongoClient

In [92]:
def parse_hh_compensation(value):
    min_com = None
    max_com = None
    cur = None
    if value:
        match = re.search(r'\b(руб|EUR|USD)\b', value)
        cur = match[0] if match else None
        
        match = re.search(r'\d*\s*\d*-\d*\s*\d*', value)
        if match:
            min_com = int(match[0].split('-')[0].replace('\xa0', ''))
            max_com = int(match[0].split('-')[1].replace('\xa0', ''))
            
        match = re.search(r'от\s\d*\s*\d*', value)
        if match:
            min_com = int(match[0].replace('\xa0', '').replace('от', '').replace(' ', ''))
            
        match = re.search(r'до\s\d*\s*\d*', value)
        if match:
            max_com = int(match[0].replace('\xa0', '').replace('до', '').replace(' ', ''))
    
    return {'min': min_com, 'max': max_com, 'cur': cur}

In [93]:
def hh_vacancy(query, page = 0):
    hh_link = 'https://hh.ru'
    params = {'st':'searchVacancy', 'text':query, 'page': page}
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    html = requests.get(hh_link+'/search/vacancy', params=params, headers=headers).text
    soup = bs(html, 'lxml')
    vacancy_block = soup.find('div', {'class':'vacancy-serp'})
    if vacancy_block is None:
        return []
    vacancy_list = vacancy_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)
    
    items = []
    for item in vacancy_list:
        data = {}
        header = item.find('div', {'class':'vacancy-serp-item__row_header'})
        compensation = header.find('div', {'class':'vacancy-serp-item__sidebar'}).find('span', {'class':'bloko-section-header-3'})
        compensation = parse_hh_compensation(compensation.getText() if compensation else None)
        data['title'] = header.find('span', {'class':'g-user-content'}).getText()
        data['compensation_min'] = compensation['min']
        data['compensation_max'] = compensation['max']
        data['compensation_cur'] = compensation['cur']
        data['link'] = header.find('a')['href']
        data['_id'] = header.find('a')['href']
        data['site'] = hh_link
        items.append(data)
    return items

In [94]:
def superjob_vacancy(query, page = 1):
    trans_query = translit(query, "ru", reversed=True).lower()
    superjob_link = 'https://russia.superjob.ru'
    offset = page * 20
    limit = 20
    params = {'page': page}
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    html = requests.get(superjob_link+'/vakansii/'+ trans_query +'.html', params=params, headers=headers).text
    soup = bs(html, 'lxml')
    vacancy_block = soup.find('div', {'class':'f-test-vacancy-item'})
    if vacancy_block is None:
        return []
    vacancy_block = vacancy_block.find_parent()
    vacancy_list = vacancy_block.findChildren('div', {'class':'f-test-vacancy-item'}, recursive=False)
    
    items = []
    for item in vacancy_list:
        data = {}        
        title = item.find('a').getText()        
        compensation = item.find('a').find_parent().find_next_sibling()
        compensation = parse_hh_compensation(compensation.getText())
        data['title'] = title
        data['compensation_min'] = compensation['min']
        data['compensation_max'] = compensation['max']
        data['compensation_cur'] = compensation['cur']
        data['link'] = superjob_link + item.find('a')['href']
        data['_id'] = superjob_link + item.find('a')['href']
        data['site'] = superjob_link
        items.append(data)
    return items

In [95]:
# Ищем вакансию в базе и если её нет, добавляем
def add_vacancy_to_db(db_vacancy, item):
    vac = db_vacancy.find_one({'_id': item['_id']})
    if vac is None:
        db_vacancy.insert_one(item)

In [96]:
query = input('Укажите вакансию для поиска: ')

client = MongoClient('localhost', 27017)
db = client['vacancy']
vacancy = db.vacancy

page = 1
while(True):
    items = hh_vacancy(query, page - 1) + superjob_vacancy(query, page)
    if len(items):
        print('Загрузка страницы ' + str(page))
        for item in items:
            add_vacancy_to_db(vacancy, item)
        page += 1
    else:
        break

Укажите вакансию для поиска: python
Загрузка страницы 1
Загрузка страницы 2
Загрузка страницы 3
Загрузка страницы 4
Загрузка страницы 5
Загрузка страницы 6
Загрузка страницы 7
Загрузка страницы 8
Загрузка страницы 9
Загрузка страницы 10
Загрузка страницы 11
Загрузка страницы 12
Загрузка страницы 13
Загрузка страницы 14
Загрузка страницы 15
Загрузка страницы 16
Загрузка страницы 17
Загрузка страницы 18
Загрузка страницы 19
Загрузка страницы 20
Загрузка страницы 21
Загрузка страницы 22
Загрузка страницы 23
Загрузка страницы 24
Загрузка страницы 25
Загрузка страницы 26
Загрузка страницы 27
Загрузка страницы 28
Загрузка страницы 29
Загрузка страницы 30
Загрузка страницы 31
Загрузка страницы 32
Загрузка страницы 33
Загрузка страницы 34
Загрузка страницы 35
Загрузка страницы 36
Загрузка страницы 37
Загрузка страницы 38
Загрузка страницы 39
Загрузка страницы 40


In [97]:
# вывод вакансий с ЗП больше указанной
def vacancy_by_qt(db_vacancy, currency, value):
    for item in db_vacancy.find({'compensation_cur': currency, "$or": [{"compensation_min": {"$gte": value}}, {"compensation_max": {"$gte": value}}]}):
        pprint(item)

In [99]:
currency = input('Укажите валюту для поиска(руб, EUR или USD): ')
value = int(input('Укажите заработную плату для поиска: '))
vacancy_by_qt(vacancy, currency, value)

Укажите валюту для поиска(руб, EUR или USD): USD
Укажите заработную плату для поиска: 2000
{'_id': 'https://bezhetsk.hh.ru/vacancy/35850257?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'compensation_cur': 'USD',
 'compensation_max': None,
 'compensation_min': 2000,
 'link': 'https://bezhetsk.hh.ru/vacancy/35850257?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'site': 'https://hh.ru',
 'title': 'Developer Drupal / Программист Drupal'}
{'_id': 'https://bezhetsk.hh.ru/vacancy/32954657?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'compensation_cur': 'USD',
 'compensation_max': None,
 'compensation_min': 2000,
 'link': 'https://bezhetsk.hh.ru/vacancy/32954657?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82',
 'site': 'https://hh.ru',
 'title': 'Ruby on Rails Developer'}
{'_id': 'https://bezhetsk.hh.ru/vacancy/35849885?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%